In [1]:
from pylab import *
from casadi import *
import time
# Required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
normalization = 1e9
δ  = 0.02
α  = 0.045007414
κ  = 2.094215255
pe = 5.42
ζ  = 1.66e-4 * normalization 
p2 =  44.75

In [3]:
#Site Data
df = pd.read_csv("data/calibration_globalModel.csv") # dataset
df = df.sort_values(by=['theta_global'])
z0_list = df['z_1995_global'].to_numpy() # initial distribution of agriculture
γ_list  = np.array([583.5109738331764]) #  distribution of gammas (aka impact of cutting down forests)
x0_list = df['x_1995_global'].to_numpy() # initial distribution of carbon absoprtion of the amazon
θ_list  = np.array([1.5200911027301107]) #  distribution of the productivity parameters

Z0_list = z0_list/ normalization
X0_list = x0_list/ normalization

z̄ = (df['zbar_2017_global'].to_numpy() )/normalization # distribution of the upper bound of agriculture in each site
n = len(z̄) # total number of sites 

In [4]:
#Construct Matrix A
Az = np.zeros((n, n+2))
Ax = np.zeros((1, n+2-0))

Ax[0,0:n-0] = -α *γ_list[0:n]
Ax[0, -1] = np.sum(α*γ_list[0:n] * z̄[0:n])
Ax[0,-2]  = -α

A  = np.concatenate((Az, Ax, np.zeros((1, n+2-0))), axis=0)

In [5]:
# Construct Matrix B
Bz = np.identity((n-0))
Bx = (np.zeros((1,n-0)))

B  = np.concatenate((Bz, Bx,  np.zeros((1, n-0))), axis=0)

In [6]:
# Construct Matrix B
Dz =   np.zeros((n-0,n-0))
Dx = -(np.ones((1,n-0))*γ_list[0:n])

D  = np.concatenate((Dz, Dx, np.zeros((1, n-0))), axis=0)

In [7]:
T   = 200 
N   = T 

dt = T/N
Y  = MX.sym('Y',n+2) 
up = MX.sym('u',n) 
um = MX.sym('u',n) 

rhs = sparsify(A)@Y + sparsify(B)@(up-um) + sparsify(D)@(up) + Y
F = Function('f', [Y, um, up],[rhs])

In [8]:

import math
ds_vect = np.zeros((N+1,1))
for i in range(N+1):
    ds_vect[i]=math.exp(-δ*i*dt)
    
pe_arrange = np.linspace(0,10,100)

In [9]:
end_z = np.zeros((100,1))
for i in range(len(pe_arrange)):
    opti = casadi.Opti()

    # Decision variables for states

    X = opti.variable(n+2-0 ,N+1)
    # Aliases for states

    Up = opti.variable(n-0,N)
    Um = opti.variable(n-0,N)
    Ua = opti.variable(1,N)
    # 1.2: Parameter for initial state
    ic = opti.parameter(n+2-0)

    # Gap-closing shooting constraints
    for k in range(N):
        opti.subject_to(X[:,k+1]==F(X[:,k],Um[:,k], Up[:,k]))

    # Initial and terminal constraints
    opti.subject_to(X[:,0] == ic)
    opti.subject_to(opti.bounded(0,X[0:n-0,:],z̄[0:n]))
    # Objective: regularization of controls
    # 1.1: added regularization
    for k in range(n-0):
        opti.subject_to(opti.bounded(0,Um[k,:],inf))
        opti.subject_to(opti.bounded(0,Up[k,:],inf))

    opti.subject_to(Ua == sum1(Up+Um)**2 )

    opti.minimize( sum2(ds_vect[0:N,:].T*(Ua* ζ/2 ))
                  - sum2(ds_vect[0:N,:].T*(pe_arrange[i]*X[-2,1:] - pe_arrange[i]*X[-2,0:-1]  ))
                  - sum2(ds_vect.T*sum1((p2*θ_list - pe_arrange[i]*κ )*X[0:n-0,:] )))
    # solve optimization problem 
    options = dict()
    options["print_time"] = False
    options["expand"]     = True
    options["ipopt"]      = {
                        'print_level': 0,
                        'fast_step_computation':            'yes',
                        'mu_allow_fast_monotone_decrease':  'yes',
                        'warm_start_init_point':            'yes',
                            }
    opti.solver('ipopt',options)

    t1 = time.time()
    opti.set_value(ic,vertcat(Z0_list,np.sum(X0_list),1))
    sol = opti.solve()
    disp(f'Initial, time: {time.time()-t1}')
    end_z[i] = sol.value(X)[0,11]



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Initial, time: 0.38709378242492676
Initial, time: 0.3850061893463135
Initial, time: 0.3926200866699219
Initial, time: 0.3252749443054199
Initial, time: 0.2930030822753906
Initial, time: 0.30234718322753906
Initial, time: 0.3380551338195801
Initial, time: 0.41382598876953125
Initial, time: 0.3388059139251709
Initial, time: 0.4168708324432373
Initial, time: 0.3882460594177246
Initial, time: 0.4035191535949707
Initial, time: 0.385009765625
Initial, time: 0.35535287857055664
Initial, time: 0.41730809211730957
Initial, time: 0.3763272762298584
Initial, time: 0.46543002128601074
Initial, time: 0.4077050685882568

In [19]:
((end_z*normalization- df['z_2017_global'].to_numpy())/df['z_2017_global'].to_numpy())[42]

array([-0.02950455])

In [20]:
pe_arrange[42]

4.242424242424242